In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

## Dataset generation

In [6]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(0.1, 0.2, 0.3).trs() * vsr.Biv(0.0,1.0,0.0).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

In [7]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
            LAs.append(((As[j].rev() * As[i]).log() * 0.5).unit())
            LBs.append(((Bs[j] * Bs[i].rev()).log() * 0.5).unit())
    return LAs, LBs

In [12]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)
print(len(lines_a))

45


In [27]:
M = vsr.Vec(1,1,1).trs() * vsr.Biv(pi/12.0, 0.0,0.0).exp()
print(M.matrix())
print(M)

[[ 0.866 -0.5    0.     1.   ]
 [ 0.5    0.866  0.     1.   ]
 [ 0.     0.     1.     1.   ]
 [ 0.     0.     0.     0.   ]]
Mot: [ 0.97 -0.26 0 0 -0.61 -0.35 -0.48 0.13 ]


In [33]:
dual_quat = lambda m : np.array([-m[3], m[2], -m[1], m[0], -m[4], -m[5], -m[6], -m[7] ])
dual_quat(M)

array([-0.    ,  0.    ,  0.2588,  0.9659,  0.6124,  0.3536,  0.483 ,
       -0.1294])

## Hand-Eye Solver

In [35]:
def daniilidis_dualquat(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        LA = LA.unit()
        LB = LB.unit()
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]
        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    
    [U, s, Vt] = np.linalg.svd(Ds)
    v7 = Vt.T[:,-2]
    v8 = Vt.T[:,-1]
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * v7 + lambda2 * v8
    m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
    m_arr[4:8] = np.roll(m_arr[4:8], -1)
    
    print(Ds)
    
    return vsr.Mot(*m_arr)

In [34]:
def daniilidis_motor(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]
        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    
    [U, s, Vt] = np.linalg.svd(Ds)
    v7 = Vt.T[:,-2]
    v8 = Vt.T[:,-1]
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * v7 + lambda2 * v8
    m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
    m_arr[4:8] = np.roll(m_arr[4:8], -1)
    
    print(Ds)
    
    return vsr.Mot(*m_arr)

## Experiments